# Check attribute before setting

In [19]:
class descriptor:
    def __init__(self, initial=0):
        self._var = initial
        
    def __get__(self, obj, owner):
        print('Calling get...')
        return self._var
    
    def __set__(self, obj, value):
        print('Calling set...')
        if value < 0:
            raise ValueError('Fuel can not be negetive')
        self._var = value
#----------------------------------------
class Car:
    fuel = descriptor(10)   # Class attribute

    def __init__(self, brand, fuel):
        self.brand = brand
        self.fuel = fuel
        
    def __repr__(self):
        return f'{self.__class__.__name__}({self.brand},{self.fuel})'

In [21]:
print("1. Instantiate an object: ", end="")
car = Car('Benz', 55)

print("\n2. Print the object: ", end="")
print(car)

print("\n3. Print the descriptor attribute: ", end="")
print(car.fuel)

print("\n4. Set the descriptor attribute: ", end="")
car.fuel = 255

print("\n5. Print the descriptor attribute: ", end="")
print(car.fuel)

print("\n6. Set the descriptor attribute to negetive value: ", end="")
car.fuel = -100

1. Instantiate an object: Calling set...

2. Print the object: Calling get...
Car(Benz,55)

3. Print the descriptor attribute: Calling get...
55

4. Set the descriptor attribute: Calling set...

5. Print the descriptor attribute: Calling get...
255

6. Set the descriptor attribute to negetive value: Calling set...


ValueError: Fuel can not be negetive

---

# Change attribute dynamicly

In [29]:
from pathlib import Path
class DirectorySizeDescriptor:
    def __get__(self, obj, owner):
        return len(list(Path(obj.dirname).iterdir()))
    

class WithDescriptorDirectory:
    size = DirectorySizeDescriptor()
    def __init__(self, dirname):
        self.dirname = dirname


class NoDescriptorDirectory:
    def __init__(self, dirname):
        self.dirname = dirname
        self.size = len(list(Path(dirname).iterdir()))

In [30]:
dir1 = WithDescriptorDirectory(".")
dir2 = NoDescriptorDirectory(".")
print("size before changing the dirname:")
print("dir1.size: ", dir1.size)
print("dir2.size: ", dir2.size)

dir1.dirname = ".."
dir2.dirname = ".."

print("size after changing the dirname:")
print("dir1.size:", dir1.size)
print("dir2.size:", dir2.size)

size before changing the dirname:
dir1.size:  5
dir2.size:  5
size after changing the dirname:
dir1.size: 16
dir2.size: 5


# Logging and set_name()

In [1]:
import logging
logging.basicConfig(filename="data/access.log", filemode="w", level=logging.INFO)

class LoggedAccessDescriptor:
    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = '_' + name
        print(
            f"Setting public and private name to {self.public_name!r} and {self.private_name!r} in the __dict__ of object: {self.__dict__}"
        )
        print("Now the attributes are stored in variables!")

    def __get__(self, obj, owner):
        value = getattr(obj, self.private_name)
        logging.info(f"Accessing '{self.public_name}' giving {value}")
        print(f"Accessing '{self.public_name}' giving {value}")
        return value

    def __set__(self, obj, value):
        logging.info(f"Updating '{self.public_name}' to {value}")
        print(f"Updating '{self.public_name}' to {value}")
        setattr(obj, self.private_name, value)


class Person:
    name = LoggedAccessDescriptor()      # First descriptor instance  (class attribute)
    age = LoggedAccessDescriptor()       # Second descriptor instance (class attribute)

    def __init__(self, name, age):
        self.name = name                 # Calls the first descriptor
        self.age = age                   # Calls the second descriptor

    def birthday(self):
        self.age += 1

Setting public and private name to 'name' and '_name' in the __dict__ of object: {'public_name': 'name', 'private_name': '_name'}
Now the attributes are stored in variables!
Setting public and private name to 'age' and '_age' in the __dict__ of object: {'public_name': 'age', 'private_name': '_age'}
Now the attributes are stored in variables!


In [10]:
print("Create an object:")
p = Person('Ali', 28)
print("\nGet the attribute:")
print(p.name)
print("\nUse a method that uses __get__ and __set__:")
p.birthday()
print("\nChecking the __dict__ of the object:")
print(p.__dict__)
print("\nChecking the __dict__ of the Class:")
print(Person.__dict__)

Create an object:
Updating 'name' to Ali
Updating 'age' to 28

Get the attribute:
Accessing 'name' giving Ali
Ali

Use a method that uses __get__ and __set__:
Accessing 'age' giving 28
Updating 'age' to 29

Checking the __dict__ of the object:
{'_name': 'Ali', '_age': 29}

Checking the __dict__ of the Class:
{'__module__': '__main__', 'name': <__main__.LoggedAccessDescriptor object at 0x7ff324150820>, 'age': <__main__.LoggedAccessDescriptor object at 0x7ff324157d30>, '__init__': <function Person.__init__ at 0x7ff315fd9ee0>, 'birthday': <function Person.birthday at 0x7ff315ff6040>, '__dict__': <attribute '__dict__' of 'Person' objects>, '__weakref__': <attribute '__weakref__' of 'Person' objects>, '__doc__': None}


---

## `getattr()` and `setattr()`
This functions are useful when the attribute name of the object is a **variable**.

`getattr(object, attribute_name)` -> return value of the attribute of that object

`setattr(object, attribute_name, value)` -> set the value for the attribute of that object 

In [34]:
class Student:
    def __init__(self, st_name):
        self.st_name = st_name

In [37]:
s = Student('Ali')
print("Print st_name attribute before changing:")
print(s.st_name)
attr_name = 'st_name'
setattr(s, attr_name, 'Khosro')
print("Print st_name attribute after changing:")
print(getattr(s, attr_name))
print("Check the __dict__ for ensurence:")
print(s.__dict__)

Print st_name attribute before changing:
Ali
Print st_name attribute after changing:
Khosro
Check the __dict__ for ensurence:
{'st_name': 'Khosro'}
